# ARIMAモデル，1次式型トレンドを含むデータ
1. ARMAシステム（真）のデータに1次式データ（非定常）が重畳する。<br>
2. そのデータの差分表示を行うと，非定常分が消えることを示す。<br>
Ref:<br>
Autoregressive Moving Average (ARMA): Artificial data https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html<br>
statsmodels.tsa.arima_model.ARIMAResults.plot_predict https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMAResults.plot_predict.html<br>

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import statsmodels.api as sm
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.arima_model import ARMA, ARIMA

from scipy import stats

np.random.seed(123)
FLAG_fig = False

観測データ（トレニーングデータ）              y  
予測精度を見るための実データ（テストデータ）  y_test

In [ ]:
ar = [1, -1.5, 0.7]
ma = [1.0, 0.6]

nobs = 1000
nobs_test = 100
nobs_all = nobs + nobs_test

dist = lambda n: np.random.randn(n)  # 正規分布,  引数 n はダミー
#dist = lambda n: np.random.standard_t(3, size=n)  # t 分布

# 知りたい信号成分
sig0_all = arma_generate_sample(ar, ma, nobs_all, sigma=1, distrvs=dist, burnin=500)

# トレンドの信号（1次式型）
coef_a, coef_b = 0.05, 4
trend0_all = coef_a*np.arange(len(sig0_all)) + coef_b

# 出力信号（トレーニングデータ＋テストデータ）
y0_all = sig0_all + trend0_all

index = pd.date_range('1/1/2000', periods=nobs_all, freq='D')
y_all = pd.Series(y0_all, index=index)

#信号だけのデータを作成
sig_all = pd.Series(sig0_all, index=index)

y = y_all[:nobs]      #観測データはｙ
y_test = y_all[nobs:] #予測精度を見るためのテストデータはy_test

y.tail(5)

y.plot(color='b')
y_test.plot(color='c')

if FLAG_fig: plt.savefig('fig_ARIMA_ident_signal_trend.png')
plt.show()

yの1階差分系列（青），元の信号（灰色）とは振幅，位相が異なることがわかる

In [ ]:
diff = (y - y.shift()).dropna(axis=0) #先頭のデータは NaNとなるため
diff.plot(color='b')
sig_all[:nobs-1].plot(color='gray')

if FLAG_fig: plt.savefig('fig_ARIMA_ident_y_diff.png')
plt.show()

ARIMAモデル,  トレンドが重畳した観測値y に対して適用

In [ ]:
arima_result = sm.tsa.ARIMA(y, order=(2,1,1)).fit(trend='nc')
print(arima_result.summary())

In [ ]:
resid = arima_result.resid # residual sequence
sig_val = 0.05 # 有意水準

resid.plot(figsize=(12,4))
print(stats.normaltest(resid))

fig = plt.figure(figsize=(12,3))
ax1 = fig.add_subplot(111)
fig = sm.graphics.tsa.plot_acf(resid.values.squeeze(), lags=20, alpha=sig_val, ax=ax1)

if FLAG_fig: plt.savefig('fig_ARIMA_ident_resid_acf.png')
plt.show()

In [ ]:
Ref:<br>
https://www.statsmodels.org/dev/examples/notebooks/generated/tsa_arma_1.html

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
fig = arima_result.plot_predict(start='2002-07-31', end='2002-10-31', ax=ax)
y_test['2002-09-27':'2002-10-31'].plot(color='m', label='real')
legend = ax.legend(loc='upper right')

if FLAG_fig: plt.savefig('fig_ARIMA_ident_y_predict.png')
plt.show()

### カーブフィッティング法  
1次式トレンドを除去する方法

In [ ]:
dummy_time = np.arange(nobs, dtype='float64')
est_a, est_b = np.polyfit(dummy_time,y,1)
print('Est a =',est_a, '    Est b=',est_b)
est_trend = est_a*np.arange(nobs, dtype='float64') + est_b

y_remove = y.sub(est_trend)
y_remove.plot(color='b')
sig_all[:nobs].plot(color='gray')

if FLAG_fig: plt.savefig('fig_ARIMA_ident_remove_trend.png')
plt.show()

In [ ]:
arma_result = sm.tsa.ARMA(y_remove, order=(2,1)).fit(trend='nc')
print(arma_result.summary())

In [ ]:
resid2 = arma_result.resid # residual sequence
resid2.plot(figsize=(12,4))
print(stats.normaltest(resid2))

fig = plt.figure(figsize=(12,3))
sig_val = 0.05 # 有意水準
ax1 = fig.add_subplot(111)
fig = sm.graphics.tsa.plot_acf(resid2.values.squeeze(), lags=20, alpha=sig_val, ax=ax1)


if FLAG_fig: plt.savefig('fig_ARIMA_ident_remove_trend_resid_acf.png')
plt.show()

予測は，ARMAの予測値に求めた一次式を重畳（単純な加算）すればよい。
しかし，statsmodelsはこのことを提供していないので，ユーザに委ねる。